In [103]:
include("packages.jl")
include("quickscore_preparation.jl")
include("quickscore_algorithm.jl")

# LOAD SOME RANDOM VARIABLES previn,pfminneg,pfmin FOR A CERTAIN m=17 and run the quickscore
file = jldopen("variables/m_18.jld", "r")
previn = file["previn"]
pfminneg = file["pfminneg"]
pfmin = file["pfmin"]
close(file)

m,n = size(pfmin)
prev = repeat(previn',inner=(n+1,1)); for i in 1:n prev[i+1,i]=1 end 
prevend = vcat(1,previn) # prevend are the prevalences 1, P(d_1=1),... P(d_n=1) and is needed to multiply at the end
prevminneg = prev.*pfminneg'

pfplus,posterior,dt = quickscore(previn, pfmin, pfminneg,false); 

m=18
flip
0: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] --> 0.0011508464813232422 
1: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] --> 0.0012598037719726562 
2: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] --> 0.0014278888702392578 
3: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] --> 0.0016829967498779297 
4: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] --> 0.0020809173583984375 
5: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] --> 0.0029408931732177734 
6: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] --> 0.004597902297973633 
7: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] --> 0.007776975631713867 
8: [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0] --> 0.021472930908203125 
9: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0] --> 0.034210920333862305 
10: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0] --> 0.06427383422851562 


11: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0] --> 0.12620997428894043 
12: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0] --> 0.2340259552001953 


13: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0] --> 0.4434068202972412 
14: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0] --> 0.8820679187774658 


15: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0] --> 1.754612922668457 
16: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0] --> 3.5190539360046387 


17: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1] --> 6.973453998565674 
18: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] --> 13.830798864364624 



Total running time was 13.830798864364624 



In [34]:
x, myset, pfmin, prevminneg, prev, M, n = rand(101, 100), [1,4,5,6], rand(18,100), rand(101,100), rand(101,100), 200000, 100
y, z, m, z_int, c = zeros(n), rand(M,n), rand(1,n), rand([-1 1],M,n), Matrix{Float64}(undef,M,1)


([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.22980037119381158 0.7053003835742215 … 0.251968697706778 0.7998111045114923; 0.6503715621173424 0.025641304708666657 … 0.11799004455748563 0.6211781509957688; … ; 0.20650915167554396 0.7212680798705308 … 0.43488155634499637 0.4651098776231325; 0.7970559295114996 0.3365055093949475 … 0.2651031660907932 0.3487389072296637], [0.4072815671488693 0.583031532646441 … 0.6072751174336536 0.38574162617301533], [1 1 … -1 1; 1 -1 … -1 -1; … ; -1 1 … -1 -1; -1 -1 … -1 1], [NaN; 2.6528509205e-314; … ; 0.0; 0.0;;])

In [153]:
setprecision(BigFloat, 20) # low precision
x = big"1.23456789"
y = sin(x)
setprecision(BigFloat, 2000) # higher precision
println(y)
x = big"1.23456789"
y = sin(x)
println(y)
